<a href="https://colab.research.google.com/github/StratosFair/Mean_Escape_Time/blob/main/error_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import re
import numpy as np
from collections import defaultdict

In [19]:
def sci_tex(x, sig=4):
    """
    Convert float to LaTeX-like string with `sig` significant figures.
    Always produces a mantissa with exactly `sig` significant digits.
    Uses scientific notation when exponent != 0.
    Examples (sig=4):
      0.0020063 -> '2.006 \\times 10^{-3}'
      0.6909    -> '6.909 \\times 10^{-1}'
      1.2345    -> '1.235'
      4.92      -> '4.920'
    """
    if x == 0:
        # e.g. sig=4 -> '0.000'
        return "0." + "0" * (sig - 1)

    # Use scientific notation with fixed total significant digits = sig
    # format: one digit before '.', (sig-1) after '.', then 'e±NN'
    s = f"{x:.{sig-1}e}"
    coeff, exp = s.split('e')
    exp = int(exp)

    if exp == 0:
        # No exponent: keep coeff, which already has sig significant digits
        # e.g. '4.920'
        return coeff

    # Non-zero exponent: mantissa × 10^{exp}
    return f"{coeff} \\times 10^{{{exp}}}"

def fmt_mean_std(values, sig=4):
    """
    Format a list of floats as 'mean±std' in LaTeX, with `sig` significant figures.
    Returns '—' if values is empty.
    """
    if not values:
        return "—"
    arr = np.array(values, dtype=float)
    mean = np.mean(arr)
    std  = np.std(arr, ddof=1) if len(arr) > 1 else 0.0
    if np.isnan(std):
        std = 0.0
    return f"{sci_tex(mean, sig)}\\pm{sci_tex(std, sig)}"

def process_file(filename, caption, sig=4):
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()

    # Full names as they should appear in the table
    order = [
        "BoundaryPINN",
        "SimplePINN",
        "VariationalPINN",
        "OverPINN $(d^{1.5})$",
        "UnderPINN $(d^{0.5})$",
    ]

    pretty_name = {
        "BoundaryPINN": "BoundaryPINN",
        "SimplePINN": "SimplePINN",
        "VariationalPINN": "VariationalPINN",
        "OverPINN $(d^{1.5})$": "OverPINN $(d^{1.5})$",
        "UnderPINN $(d^{0.5})$": "UnderPINN $(d^{0.5})$",
    }

    # Map a simple base name (as appears in \subsection) to the full key
    base_to_arch = {
        "BoundaryPINN": "BoundaryPINN",
        "SimplePINN": "SimplePINN",
        "VariationalPINN": "VariationalPINN",
        "OverPINN": "OverPINN $(d^{1.5})$",
        "UnderPINN": "UnderPINN $(d^{0.5})$",
    }

    # Data structure: data[architecture][metric] -> list of floats
    data = defaultdict(lambda: defaultdict(list))

    # Phrases to look for in each line
    patterns = {
        "L2":   "Global relative L2 error",
        "L1":   "Global relative L1 error",
        "H1":   "Global relative H1 error",
        "H2":   "Global relative H2 error",
        "PDE":  "Final PDE loss",
        "BC":   "Final BC loss",
        "Data": "Final data loss",
    }

    current_arch = None

    # ---- PARSING LOOP ----
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # Section headers identify the current architecture
        if line.startswith("\\subsection"):
            current_arch = None
            # Remove braces so things like ^{1.5} become ^1.5, etc.
            clean = line.replace("{", "").replace("}", "")
            for base, arch in base_to_arch.items():
                if base in clean:
                    current_arch = arch
                    break
            continue

        if current_arch is None:
            continue

        # Match metrics by phrase and then capture the number just after the colon
        for key, phrase in patterns.items():
            if phrase in line:
                match = re.search(r":\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)", line)
                if match:
                    val = float(match.group(1))
                    data[current_arch][key].append(val)
                break

    # ---- ERROR METRICS TABLE (L2, H1, H2) ----
    print("\\begin{table}[htbp]")
    print("\t\\centering")
    print(f"\t\\caption{{{caption} -- Error metrics}}")
    print("\t\\small")
    print("\t\\begin{tabular}{lccc}")
    print("\t\t\\toprule")
    print("\t\tMethod & $L^2_{\\rm rel}$ & $H^1_{\\rm rel}$ & $H^2_{\\rm rel}$ \\\\")
    print("\t\t\\midrule")

    for arch in order:
        row = [pretty_name[arch]]
        # Each numeric entry wrapped in math mode
        row.append(f"${fmt_mean_std(data[arch]['L2'], sig)}$")
        row.append(f"${fmt_mean_std(data[arch]['H1'], sig)}$")
        row.append(f"${fmt_mean_std(data[arch]['H2'], sig)}$")
        print("\t\t" + " & ".join(row) + " \\\\")

    print("\t\t\\bottomrule")
    print("\t\\end{tabular}")
    print("\\end{table}")
    print()

    # ---- TRAINING METRICS TABLE (PDE, BC, Data) ----
    print("\\begin{table}[htbp]")
    print("\t\\centering")
    print(f"\t\\caption{{{caption} -- Training metrics}}")
    print("\t\\small")
    print("\t\\begin{tabular}{lccc}")
    print("\t\t\\toprule")
    print("\t\tMethod & PDE loss & BC loss & Data loss \\\\")
    print("\t\t\\midrule")

    for arch in order:
        row = [pretty_name[arch]]
        row.append(f"${fmt_mean_std(data[arch]['PDE'], sig)}$")
        row.append(f"${fmt_mean_std(data[arch]['BC'], sig)}$")
        row.append(f"${fmt_mean_std(data[arch]['Data'], sig)}$")
        print("\t\t" + " & ".join(row) + " \\\\")

    print("\t\t\\bottomrule")
    print("\t\\end{tabular}")
    print("\\end{table}")

In [20]:
process_file("ou.txt", "Ornstein-Uhlenbeck 2D", sig=4)

\begin{table}[htbp]
	\centering
	\caption{Ornstein-Uhlenbeck 2D -- Error metrics}
	\small
	\begin{tabular}{lccc}
		\toprule
		Method & $L^2_{\rm rel}$ & $H^1_{\rm rel}$ & $H^2_{\rm rel}$ \\
		\midrule
		BoundaryPINN & $2.006 \times 10^{-3}\pm3.014 \times 10^{-4}$ & $1.998 \times 10^{-3}\pm2.981 \times 10^{-4}$ & $1.990 \times 10^{-3}\pm2.946 \times 10^{-4}$ \\
		SimplePINN & $4.887 \times 10^{-2}\pm3.872 \times 10^{-2}$ & $4.994 \times 10^{-2}\pm3.926 \times 10^{-2}$ & $5.041 \times 10^{-2}\pm3.964 \times 10^{-2}$ \\
		VariationalPINN & $1.169 \times 10^{-1}\pm5.697 \times 10^{-2}$ & $6.491 \times 10^{-1}\pm6.106 \times 10^{-1}$ & $3.269\pm4.211$ \\
		OverPINN $(d^{1.5})$ & $1.013\pm5.774 \times 10^{-4}$ & $1.006\pm0.000$ & $1.001\pm5.774 \times 10^{-4}$ \\
		UnderPINN $(d^{0.5})$ & $1.009\pm1.732 \times 10^{-3}$ & $1.005\pm5.774 \times 10^{-4}$ & $1.000\pm0.000$ \\
		\bottomrule
	\end{tabular}
\end{table}

\begin{table}[htbp]
	\centering
	\caption{Ornstein-Uhlenbeck 2D -- Training met

In [21]:
process_file("dw.txt", "Double Well 2D")

\begin{table}[htbp]
	\centering
	\caption{Double Well 2D -- Error metrics}
	\small
	\begin{tabular}{lccc}
		\toprule
		Method & $L^2_{\rm rel}$ & $H^1_{\rm rel}$ & $H^2_{\rm rel}$ \\
		\midrule
		BoundaryPINN & $2.643 \times 10^{-1}\pm3.512 \times 10^{-4}$ & $3.233 \times 10^{-1}\pm8.386 \times 10^{-4}$ & $4.031 \times 10^{-1}\pm1.300 \times 10^{-3}$ \\
		SimplePINN & $2.953 \times 10^{-1}\pm1.200 \times 10^{-2}$ & $4.499 \times 10^{-1}\pm1.393 \times 10^{-2}$ & $6.530 \times 10^{-1}\pm2.366 \times 10^{-2}$ \\
		VariationalPINN & $5.639 \times 10^{-1}\pm1.100 \times 10^{-2}$ & $1.173\pm9.637 \times 10^{-2}$ & $1.197 \times 10^{1}\pm1.703$ \\
		OverPINN $(d^{1.5})$ & $4.857 \times 10^{-1}\pm3.981 \times 10^{-1}$ & $5.865 \times 10^{-1}\pm4.187 \times 10^{-1}$ & $8.943 \times 10^{-1}\pm2.767 \times 10^{-1}$ \\
		UnderPINN $(d^{0.5})$ & $9.047 \times 10^{-1}\pm8.524 \times 10^{-3}$ & $9.202 \times 10^{-1}\pm9.625 \times 10^{-3}$ & $9.625 \times 10^{-1}\pm8.305 \times 10^{-3}$ \\
		\bottom